<a href="https://colab.research.google.com/github/profteachkids/CHE5136_Fall2023/blob/main/max_entropy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import jax
jax.config.update('jax_enable_x64',True)
import jax.numpy as jnp
from plotly.subplots import make_subplots
from scipy.optimize import minimize
import scipy as sp

In [2]:
Np = 51
x = np.arange(Np)
p_guess = np.full(Np, 1./Np)

In [3]:
@jax.jit
def neg_entropy(p):
    return jnp.sum(p*jnp.log(p))

jac_neg_entropy = jax.jit(jax.jacobian(neg_entropy))

In [46]:
@jax.jit
def psum1(p):
    return jnp.sum(p)-1.
jac_psum1 = jax.jit(jax.jacobian(psum1))

@jax.jit
def avg_constraint(p):
    return jnp.sum(p*x) - 36.5
jac_avg_constraint = jax.jit(jax.jacobian(avg_constraint))

@jax.jit
def std_constraint(p):
    avg = jnp.sum(p*x)
    return jnp.sqrt( jnp.sum(p* ((x-avg)**2))) - 6.5
jac_std_constraint = jax.jit(jax.jacobian(std_constraint))

@jax.jit
def fail_constraint(p):
    return jnp.sum(p[:(Np//2)]) - 0.2
jac_fail_constraint = jax.jit(jax.jacobian(fail_constraint))

In [47]:
res=minimize(neg_entropy, p_guess, jac =jac_neg_entropy, method='SLSQP',
         bounds = ((0,1),)*Np, options=dict(maxiter=5000),
             constraints=[
              dict(type='eq', fun=psum1, jac=jac_psum1),
              dict(type='eq', fun=avg_constraint, jac=jac_avg_constraint),
              dict(type='eq', fun=std_constraint, jac=jac_std_constraint),
              dict(type='ineq', fun=fail_constraint, jac=jac_fail_constraint)])
res

 message: Optimization terminated successfully
 success: True
  status: 0
     fun: -2.1731348242704485
       x: [ 4.691e-10  4.572e-10 ...  7.479e-06  4.220e-06]
     nit: 171
     jac: [-2.048e+01 -2.051e+01 ... -1.080e+01 -1.138e+01]
    nfev: 352
    njev: 171

In [48]:
fig=make_subplots()
fig.add_bar(x=x, y=res.x)
fig.update_layout(height=600, width=800, template='plotly_dark')